In [1]:
!export PYTHONPATH=.:$PYTHONPATH

In [2]:
from odi.feature_engg import feature_extractor as fe
from odi.feature_engg import util as cricutil
from odi.preprocessing import rank
import pandas as pd
from datetime import datetime
import math
import numpy as np
from tqdm.notebook import tqdm

In [3]:
match_list_df=cricutil.read_csv_with_date('data/csv_load/cricinfo_match_list.csv')
batting_df=cricutil.read_csv_with_date('data/csv_load/cricinfo_batting.csv')

In [4]:
start_dt = cricutil.str_to_date_time("2011-01-01")
end_dt = cricutil.str_to_date_time("2019-01-01")

In [5]:
match_list_df = match_list_df[(match_list_df['date']>=start_dt) & (match_list_df['date']<end_dt)]

In [6]:
match_id_list = list(match_list_df['match_id'].unique())

In [61]:
broke=False
dict_list = []
for match_id in tqdm(match_id_list):
    for innings_type in ['first','second']:
        runs = match_list_df[match_list_df["match_id"]==match_id].iloc[0][innings_type+"_innings_run"]
        team = match_list_df[match_list_df["match_id"]==match_id].iloc[0][innings_type+"_innings"]
        date = match_list_df['date'].iloc[0]
        ref_date = date.to_pydatetime()
        player_list_df = batting_df[(batting_df['match_id']==match_id) & (batting_df['batting_innings']==innings_type)]
        
        #print("ref_date ",ref_date)
        #print("initial player list ",player_list_df.shape)
        latest_batsman_rank_file = rank.get_latest_rank_file("batsman",ref_date=ref_date)
        #print("latest_batsman_rank_file",latest_batsman_rank_file)
        batsman_rank_df = pd.read_csv(latest_batsman_rank_file)
        
        player_list_df = player_list_df[['team','name','position']]
        batsman_rank_df = batsman_rank_df[['batsman','country','winning_contribution','run_rate_effectiveness','correlation','batsman_score','batsman_quantile']]
        batsman_rank_df.rename(columns={'batsman':'name','country':'team'},inplace=True)
        
        player_list_df = player_list_df.merge(batsman_rank_df,how='left',on=['name','team'])
        #print("player list after merging",player_list_df.shape)
#         if player_list_df.shape[0]<11:
#             continue
        
#         if len(player_list_df[player_list_df['batsman_score'].isnull()].index)>0:
#             indices = list(player_list_df[player_list_df['batsman_score'].isnull()].index)
#             skip = False
#             for ind in indices:
#                 if player_list_df.loc[ind].position<=7:
#                     skip = True
#                     break
#             if skip == True:
#                 continue
#             else:
#                 player_list_df.dropna(inplace=True)
        player_list_df = player_list_df.head(7)
        if len(player_list_df[player_list_df['batsman_score'].isnull()].index)>2:
            continue
        elif len(player_list_df[player_list_df['batsman_score'].isnull()].index)>0:
             
            mean_score = player_list_df[~(player_list_df['batsman_score'].isnull())]['batsman_score'].mean()
            mean_correlation = player_list_df[~(player_list_df['correlation'].isnull())]['correlation'].mean()
            mean_run_rate_effectiveness = player_list_df[~(player_list_df['run_rate_effectiveness'].isnull())]['run_rate_effectiveness'].mean()
            mean_winning_contribution = player_list_df[~(player_list_df['winning_contribution'].isnull())]['winning_contribution'].mean()
            
            missing_indices = player_list_df[player_list_df['batsman_score'].isnull()].index
            for missing_index in missing_indices:
                player_list_df.loc[missing_index,'batsman_score']=mean_score
                player_list_df.loc[missing_index,'mean_correlation']=mean_correlation
                player_list_df.loc[missing_index,'mean_run_rate_effectiveness']=mean_run_rate_effectiveness
                player_list_df.loc[missing_index,'mean_winning_contribution']=mean_winning_contribution
        else:
            pass
            
        player_list_df['position']=player_list_df['position'].astype(int)
        player_list_df.sort_values(['position'],inplace=True)
        
        score_sum_7 = player_list_df.head(7)['batsman_score'].sum()
        score_mean_7 = player_list_df.head(7)['batsman_score'].mean()
        
        score_sum_6 = player_list_df.head(6)['batsman_score'].sum()
        score_mean_6 = player_list_df.head(6)['batsman_score'].mean()
        
        score_sum_5 = player_list_df.head(5)['batsman_score'].sum()
        score_mean_5 = player_list_df.head(5)['batsman_score'].mean()
        
        score_sum_4 = player_list_df.head(4)['batsman_score'].sum()
        score_mean_4 = player_list_df.head(4)['batsman_score'].mean()
        
        score_sum_3 = player_list_df.head(3)['batsman_score'].sum()
        score_mean_3 = player_list_df.head(3)['batsman_score'].mean()
        
        score_max = player_list_df.head(7)['batsman_score'].max()
        
        # weighted_by_correlation
        player_list_df['score_weighted_corr']=player_list_df['batsman_score']*player_list_df['correlation']
        player_list_df['score_weighted_contr']=player_list_df['batsman_score']*player_list_df['winning_contribution']
        player_list_df['score_weighted_effectiveness']=player_list_df['batsman_score']*player_list_df['run_rate_effectiveness']
        player_list_df['log_factor'] = 13-player_list_df['position']
        #player_list_df.fillna(1,inplace=True)
        player_list_df['log_scale'] = np.log(player_list_df['log_factor'])
        player_list_df['score_weighted_log'] = player_list_df['batsman_score']*player_list_df['log_scale']
        
        score_sum_weighted_by_correlation = player_list_df.head(7)['score_weighted_corr'].sum()
        score_sum_weighted_by_contribution = player_list_df.head(7)['score_weighted_contr'].sum()
        score_sum_weighted_by_effectiveness = player_list_df.head(7)['score_weighted_effectiveness'].sum()
        score_sum_weighted_by_log = player_list_df.head(7)['score_weighted_log'].sum()
        
        score_mean_weighted_by_correlation = player_list_df.head(7)['score_weighted_corr'].sum()/player_list_df.head(7)['correlation'].sum()
        score_mean_weighted_by_contribution = player_list_df.head(7)['score_weighted_contr'].sum()/player_list_df.head(7)['winning_contribution'].sum()
        score_mean_weighted_by_effectiveness = player_list_df.head(7)['score_weighted_effectiveness'].sum()/player_list_df.head(7)['run_rate_effectiveness'].sum()
        score_mean_weighted_by_log = player_list_df.head(7)['score_weighted_log'].sum()/player_list_df.head(7)['log_scale'].sum()
        
        score_sum_weighted_by_correlation_4 = player_list_df.head(4)['score_weighted_corr'].sum()
        score_sum_weighted_by_contribution_4 = player_list_df.head(4)['score_weighted_contr'].sum()
        score_sum_weighted_by_effectiveness_4 = player_list_df.head(4)['score_weighted_effectiveness'].sum()
        score_sum_weighted_by_log_4 = player_list_df.head(4)['score_weighted_log'].sum()
        
        score_mean_weighted_by_correlation_4 = player_list_df.head(4)['score_weighted_corr'].sum()/player_list_df.head(4)['correlation'].sum()
        score_mean_weighted_by_contribution_4 = player_list_df.head(4)['score_weighted_contr'].sum()/player_list_df.head(4)['winning_contribution'].sum()
        score_mean_weighted_by_effectiveness_4 = player_list_df.head(4)['score_weighted_effectiveness'].sum()/player_list_df.head(4)['run_rate_effectiveness'].sum()
        score_mean_weighted_by_log_4 = player_list_df.head(4)['score_weighted_log'].sum()/player_list_df.head(4)['log_scale'].sum()
        
        
        player_1 = player_list_df['batsman_score'].values[0]
        player_2 = player_list_df['batsman_score'].values[1]
        player_3 = player_list_df['batsman_score'].values[2]
        player_4 = player_list_df['batsman_score'].values[3]
        player_5 = player_list_df['batsman_score'].values[4]
        player_6 = player_list_df['batsman_score'].values[5]
        player_7 = player_list_df['batsman_score'].values[6]
        
        entry = {
            "score_sum_7":score_sum_7,
            "score_sum_6":score_sum_6,
            "score_sum_5":score_sum_5,
            "score_sum_4":score_sum_4,
            "score_sum_3":score_sum_3,
            "score_max":score_max,
            "score_sum_weighted_by_correlation":score_sum_weighted_by_correlation,
            "score_sum_weighted_by_contribution":score_sum_weighted_by_contribution,
            "score_sum_weighted_by_effectiveness":score_sum_weighted_by_effectiveness,
            "score_sum_weighted_by_log":score_sum_weighted_by_log,
            "score_mean_weighted_by_correlation":score_mean_weighted_by_correlation,
            "score_mean_weighted_by_contribution":score_mean_weighted_by_contribution,
            "score_mean_weighted_by_effectiveness":score_mean_weighted_by_effectiveness,
            "score_mean_weighted_by_log":score_mean_weighted_by_log,
            "score_sum_weighted_by_correlation_4":score_sum_weighted_by_correlation_4,
            "score_sum_weighted_by_contribution_4":score_sum_weighted_by_contribution_4,
            "score_sum_weighted_by_effectiveness_4":score_sum_weighted_by_effectiveness_4,
            "score_sum_weighted_by_log_4":score_sum_weighted_by_log_4,
            "score_mean_weighted_by_correlation_4":score_mean_weighted_by_correlation_4,
            "score_mean_weighted_by_contribution_4":score_mean_weighted_by_contribution_4,
            "score_mean_weighted_by_effectiveness_4":score_mean_weighted_by_effectiveness_4,
            "score_mean_weighted_by_log_4":score_mean_weighted_by_log_4,
            "score_mean_7":score_mean_7,
            "score_mean_6":score_mean_6,
            "score_mean_5":score_mean_5,
            "score_mean_4":score_mean_4,
            "score_mean_3":score_mean_3,
            "player_1":player_1,
            "player_2":player_2,
            "player_3":player_3,
            "player_4":player_4,
            "player_5":player_5,
            "player_6":player_6,
            "player_7":player_7,
            "innings_type":innings_type,
            "runs":runs
          
        }
        
        dict_list.append(entry)
#         print(innings_type)
#         print(team)
#         print(runs)
#         print(list(player_list_df['name']))
        
#         print("============")
        
    #break
#     if broke:
#         break
        
        
        
        
        
        
        
        
        
        
        

/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/ipykernel_launcher.py:101: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensor230/lib/python3.6/site-packages/ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in double_scalars


In [62]:
from scipy.stats import pearsonr

In [63]:
score_df = pd.DataFrame(dict_list)

In [64]:
score_df.shape,len(match_id_list)

((546, 36), 622)

In [65]:
score_df

,score_sum_7,score_sum_6,score_sum_5,score_sum_4,score_sum_3,score_max,score_sum_weighted_by_correlation,score_sum_weighted_by_contribution,score_sum_weighted_by_effectiveness,score_sum_weighted_by_log,...,score_mean_3,player_1,player_2,player_3,player_4,player_5,player_6,player_7,innings_type,runs
0,37.363380,33.624010,30.055771,25.583904,17.408991,10.000000,11.464309,5.580787,33.605197,82.652158,...,5.802997,3.887324,10.000000,3.521667,8.174914,4.471866,3.568239,3.739370,first,289.0
1,50.701704,43.651046,36.254791,30.144521,23.471736,9.998766,21.878933,4.537273,47.816052,110.171254,...,7.823912,5.153177,8.319793,9.998766,6.672784,6.110271,7.396255,7.050659,second,154.0
2,50.701704,44.028920,36.978262,29.582007,23.471736,9.998766,21.878933,4.537273,47.816052,110.259855,...,7.823912,5.153177,8.319793,9.998766,6.110271,7.396255,7.050659,6.672784,first,190.0
3,37.363380,33.624010,30.055771,25.583904,17.408991,10.000000,11.464309,5.580787,33.605197,83.184031,...,5.802997,10.000000,3.887324,3.521667,8.174914,4.471866,3.568239,3.739370,second,189.0
4,41.401687,37.249199,28.629332,24.059096,19.446285,8.628975,12.031597,4.448828,38.451105,90.739970,...,6.482095,8.628975,5.637679,5.179631,4.612812,4.570236,8.619866,4.152489,first,294.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,46.875160,41.085820,33.689566,26.993114,20.296663,9.998766,13.258006,3.187349,26.439177,101.700934,...,6.765554,6.672784,3.625112,9.998766,6.696451,6.696451,7.396255,5.789340,second,326.0
542,46.875160,41.085820,34.389369,26.993114,20.296663,9.998766,13.258006,3.187349,26.439177,101.794380,...,6.765554,6.672784,3.625112,9.998766,6.696451,7.396255,6.696451,5.789340,first,377.0
543,46.875160,41.085820,33.689566,26.993114,20.296663,9.998766,13.258006,3.187349,26.439177,101.700934,...,6.765554,6.672784,3.625112,9.998766,6.696451,6.696451,7.396255,5.789340,second,105.0
544,39.619883,34.953474,29.293491,22.114751,17.543811,7.178740,4.839770,3.715108,21.048392,86.302336,...,5.847937,5.598415,5.659983,6.285412,4.570941,7.178740,5.659983,4.666409,second,196.0


In [66]:
score_df.isnull().sum()

score_sum_7                                0
score_sum_6                                0
score_sum_5                                0
score_sum_4                                0
score_sum_3                                0
score_max                                  0
score_sum_weighted_by_correlation          0
score_sum_weighted_by_contribution         0
score_sum_weighted_by_effectiveness        0
score_sum_weighted_by_log                  0
score_mean_weighted_by_correlation         0
score_mean_weighted_by_contribution       37
score_mean_weighted_by_effectiveness      37
score_mean_weighted_by_log                 0
score_sum_weighted_by_correlation_4        0
score_sum_weighted_by_contribution_4       0
score_sum_weighted_by_effectiveness_4      0
score_sum_weighted_by_log_4                0
score_mean_weighted_by_correlation_4       0
score_mean_weighted_by_contribution_4     37
score_mean_weighted_by_effectiveness_4    37
score_mean_weighted_by_log_4               0
score_mean

In [67]:
score_df.dropna(inplace=True)

In [68]:
all_cols = list(score_df.columns)


In [69]:
#score_df.dropna(inplace=True)

In [70]:
target = list(score_df['runs'])
displaye_list = []
for col_idx in range(len(all_cols)-2):
    l = list(score_df[all_cols[col_idx]])
    corr,p_value = pearsonr(l, target)
    #print(all_cols[col_idx],'\t\t\t\t',corr,'\t',p_value)
    display_dict = {
        'col':all_cols[col_idx],
        'cor':corr,
        'p_value':p_value
    }
    
    displaye_list.append(display_dict)

In [71]:
#all_cols[col_idx]

In [72]:
score_importance_overall = pd.DataFrame(displaye_list).sort_values(['cor'],ascending=False)
score_importance_overall

,col,cor,p_value
11,score_mean_weighted_by_contribution,0.203727,0.000004
19,score_mean_weighted_by_contribution_4,0.196589,0.000008
20,score_mean_weighted_by_effectiveness_4,0.180218,0.000043
30,player_4,0.174497,0.000076
5,score_max,0.161055,0.000264
12,score_mean_weighted_by_effectiveness,0.159883,0.000293
3,score_sum_4,0.149413,0.000721
25,score_mean_4,0.149413,0.000721
21,score_mean_weighted_by_log_4,0.146733,0.000899
17,score_sum_weighted_by_log_4,0.146733,0.000899


In [73]:
score_df_first = score_df[score_df['innings_type']=='first']
target = list(score_df_first['runs'])
displaye_list_first = []
for col_idx in range(len(all_cols)-2):
    l = list(score_df_first[all_cols[col_idx]])
    corr,p_value = pearsonr(l, target)
    #print(all_cols[col_idx],'\t\t\t\t',corr,'\t',p_value)
    display_dict = {
        'col':all_cols[col_idx],
        'cor':corr,
        'p_value':p_value
    }
    
    displaye_list_first.append(display_dict)

In [74]:
score_importance_first = pd.DataFrame(displaye_list_first).sort_values(['cor'],ascending=False)
score_importance_first

,col,cor,p_value
11,score_mean_weighted_by_contribution,0.200457,0.001542
19,score_mean_weighted_by_contribution_4,0.195231,0.002053
5,score_max,0.178950,0.004788
30,player_4,0.177209,0.005221
20,score_mean_weighted_by_effectiveness_4,0.160912,0.011321
12,score_mean_weighted_by_effectiveness,0.134264,0.034947
32,player_6,0.123575,0.052412
29,player_3,0.116646,0.067222
3,score_sum_4,0.111537,0.080203
25,score_mean_4,0.111537,0.080203


In [75]:
score_df_second = score_df[score_df['innings_type']=='second']
target = list(score_df_second['runs'])
displaye_list_second = []
for col_idx in range(len(all_cols)-2):
    l = list(score_df_second[all_cols[col_idx]])
    corr,p_value = pearsonr(l, target)
    #print(all_cols[col_idx],'\t\t\t\t',corr,'\t',p_value)
    display_dict = {
        'col':all_cols[col_idx],
        'cor':corr,
        'p_value':p_value
    }
    
    displaye_list_second.append(display_dict)

In [76]:
score_importance_second = pd.DataFrame(displaye_list_second).sort_values(['cor'],ascending=False)
score_importance_second

,col,cor,p_value
11,score_mean_weighted_by_contribution,0.214521,0.000471
20,score_mean_weighted_by_effectiveness_4,0.213816,0.000493
12,score_mean_weighted_by_effectiveness,0.212160,0.000546
19,score_mean_weighted_by_contribution_4,0.201473,0.001041
25,score_mean_4,0.198917,0.001209
3,score_sum_4,0.198917,0.001209
21,score_mean_weighted_by_log_4,0.196485,0.001392
17,score_sum_weighted_by_log_4,0.196485,0.001392
0,score_sum_7,0.193353,0.001664
22,score_mean_7,0.193353,0.001664


In [77]:
# for col_idx in range(len(all_cols)-2):
#     score_df[[all_cols[col_idx]]].plot.kde()

In [58]:
list(score_importance_second['col'][:4])

['score_mean_weighted_by_effectiveness',
 'score_mean_weighted_by_contribution',
 'score_mean_weighted_by_effectiveness_4',
 'score_sum_7']

In [59]:
match_id

4073

In [60]:
player_list_df = batting_df[(batting_df['match_id']==match_id) & (batting_df['batting_innings']==innings_type)]
player_list_df

,match_id,team,opponent,batting_innings,date,did_bat,player_of_the_match,is_captain,wc,name,position,is_out,wicket_type,runs,balls,m,4s,6s,sr
17545,4073,Bangladesh,West Indies,second,2018-12-14,1,0,0,0,Tamim Iqbal,1.0,0.0,not out,81,104,0,9,0,77.88
17546,4073,Bangladesh,West Indies,second,2018-12-14,1,0,0,0,Liton Das,2.0,1.0,c Powell b Paul,23,33,0,5,0,69.69
17547,4073,Bangladesh,West Indies,second,2018-12-14,1,0,0,0,Soumya Sarkar,3.0,1.0,b Paul,80,81,0,5,5,98.76
17548,4073,Bangladesh,West Indies,second,2018-12-14,1,0,0,1,Mushfiqur Rahim,4.0,0.0,not out,16,14,0,2,1,114.28
17549,4073,Bangladesh,West Indies,second,2018-12-14,0,0,0,0,Mohammad Mithun,5.0,NaN,NaN,0,0,0,0,0,0.00
17550,4073,Bangladesh,West Indies,second,2018-12-14,0,0,0,0,Shakib Al Hasan,6.0,NaN,NaN,0,0,0,0,0,0.00
17551,4073,Bangladesh,West Indies,second,2018-12-14,0,0,0,0,Mahmudullah,7.0,NaN,NaN,0,0,0,0,0,0.00
17552,4073,Bangladesh,West Indies,second,2018-12-14,0,0,1,0,Mashrafe Mortaza,8.0,NaN,NaN,0,0,0,0,0,0.00
17553,4073,Bangladesh,West Indies,second,2018-12-14,0,1,0,0,Mehidy Hasan Miraz,9.0,NaN,NaN,0,0,0,0,0,0.00
17554,4073,Bangladesh,West Indies,second,2018-12-14,0,0,0,0,Mohammad Saifuddin,10.0,NaN,NaN,0,0,0,0,0,0.00


In [79]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression